# Sepsis Project: (main.ipynb)

# Setup

## Environment

In [ ]:
## You can run this notebook also in your local VS-code ##
# =====================================
# 1) Setup: Detect Colab and set project folder
# =====================================

import sys
import os

# Detect if running in Colab
is_colab = 'google.colab' in sys.modules
print("Running in Colab?", is_colab)

# If in Colab, mount Drive and set path
if is_colab:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    #PROJECT_PATH = '/content/drive/MyDrive/Deep Learning S25 Course Project'
    PROJECT_PATH = '/content/drive/MyDrive/Deep Learning S25 Course Project'
else:
    # Local dev: use current folder or adjust if needed
    PROJECT_PATH = os.getcwd()

# Change working directory
os.chdir(PROJECT_PATH)

# Add to sys.path for custom imports
if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())

# Confirm
print("Current working directory:", os.getcwd())
print("sys.path includes this folder:", os.getcwd() in sys.path)

# Confirm contents
print("\nFolder contents:")
for item in os.listdir():
    print("-", item)

print("\nData folder contents:")
print(os.listdir("data"))

print("\nData_Preparation folder contents:")
print(os.listdir("Data_Preparation"))


In [ ]:
# =====================================
# 2) Check what's in your data folder
# =====================================
print("Data folder files:", os.listdir("data"))


In [ ]:
# Installing requirements
# %pip install -q \
#     imbalanced-learn \
#     imblearn \
#     matplotlib \
#     numpy \
#     pandas \
#     scikit-learn \
#     seaborn \
#     torch \
#     tqdm \
#     ipywidgets \
#     notebook \
#     joblib

## Data Import

In [ ]:
# Whether data preprocessing step should be computed again
# If false, load previously saved preprocessed data
RECOMPUTE_DATA_PREPROCESSING = True
LOAD_CLEAN_DATA = False

# Whether to load raw data for EDA
RUN_EDA_AB = False
RUN_EDA_A = False
RUN_EDA_B = False 

In [ ]:
# =====================================
# 3) Data Import and structure
# =====================================
import pandas as pd

# Correct path: use 'data/' now
DATA_FILE_AB = 'data/raw/training_set_AB.csv'
DATA_FILE_A = 'data/raw/training_set_A.csv'
DATA_FILE_B = 'data/raw/training_set_B.csv'


# training_set_A: Data from Hospital System A ========>  data_A (Use for training)
# training_set_B: Data from Hospital System B ========>  data_B (Use for validation or testing)
# training_set_AB: Combined Data from Hospital System A and B ========> (We probably won't use it) - We still can use it in EDA
data_AB = None
data_A = None
data_B = None

if RECOMPUTE_DATA_PREPROCESSING or RUN_EDA_AB:
  data_AB = pd.read_csv(DATA_FILE_AB)

if RECOMPUTE_DATA_PREPROCESSING or RUN_EDA_A:
  data_A = pd.read_csv(DATA_FILE_A)

if RECOMPUTE_DATA_PREPROCESSING or RUN_EDA_B:
  data_B = pd.read_csv(DATA_FILE_B)

# Drop columns with 60%+ missingness
row_ct = None if data_AB is None else data_AB.shape[0]
threshold = None if data_AB is None else int(row_ct * 0.2)
data_AB_cleaned = None if data_AB is None else data_AB.dropna(axis=1, thresh=threshold)
if not data_AB_cleaned is None:
  print(f'Kept {len(data_AB_cleaned.columns.to_list()) - 3} feature columns.')


In [ ]:
'Skipped loading raw data!' if data_AB is None else data_AB.head()

# Data Preparation

## Label generation & dataset splitting

In [ ]:
# (Asal) Run Label generation & dataset splitting

#!ls -l
from Data_Preparation import(
  stratified_group_k_fold
)

splits = None

if RECOMPUTE_DATA_PREPROCESSING:
  splits = stratified_group_k_fold(data_AB_cleaned, k=5)

  for fold, (train_idx, test_idx) in enumerate(splits):
      train_df = data_AB.iloc[train_idx]
      test_df = data_AB.iloc[test_idx]

      # Count positive labels
      train_pos = train_df['SepsisLabel'].sum()
      test_pos = test_df['SepsisLabel'].sum()

      # Count total labels
      train_total = len(train_df)
      test_total = len(test_df)

      print(f"\nFold {fold+1}")
      print(f"Train size: {train_total}, Positive cases: {train_pos} ({100 * train_pos / train_total:.2f}%)")
      print(f"Test   size: {test_total}, Positive cases: {test_pos} ({100 * test_pos / test_total:.2f}%)")

else:
  print("Skipping data preprocessing!")


## Missing Value Imputation

In [ ]:
# (Mitch) Run Data parsing and Handling missing data codes here
from Data_Preparation import parse_and_clean_data
from pathlib import Path
from tqdm.notebook import tqdm
import pandas as pd

CLEANED_DATA_DIR = Path('data/cleaned/')
if not CLEANED_DATA_DIR.is_dir():
  CLEANED_DATA_DIR.mkdir()

#Strategy for handling missing values
IMPUTE = 'impute'
MASK = 'mask'
MASK_IMPUTE = 'mask-impute'

MISSING_VAL_STRATEGY = IMPUTE # 'mask' 'impute' 'mask-impute'
STRATEGIES_TO_LOAD = [MISSING_VAL_STRATEGY] # add any other strategies of interest to load them into the data_dict

TRAIN = 'train'
TEST = 'test'
data_dict = {}

if RECOMPUTE_DATA_PREPROCESSING:
  print("Preprocessing data!")
  for fold, (train_idx, test_idx) in enumerate(splits):
    for strategy in STRATEGIES_TO_LOAD:
      train_df = data_AB_cleaned.iloc[train_idx]
      test_df = data_AB_cleaned.iloc[test_idx]
      train_df_clean = parse_and_clean_data(df=train_df, missing_values=strategy)
      test_df_clean = parse_and_clean_data(df=test_df, missing_values=strategy)
      data_dict[fold] = {}
      data_dict[fold][TRAIN] = {}
      data_dict[fold][TEST] = {}
      data_dict[fold][TRAIN][strategy] = train_df_clean
      data_dict[fold][TEST][strategy] = test_df_clean

      train_fname = "_".join((str(fold), TRAIN, strategy))
      train_fname = ".".join((train_fname, "csv"))
      train_df_clean.to_csv(CLEANED_DATA_DIR.joinpath(train_fname), index=False)

      test_fname = "_".join((str(fold), TEST, strategy))
      test_fname = ".".join((test_fname, "csv"))
      test_df_clean.to_csv(CLEANED_DATA_DIR.joinpath(test_fname), index=False)
elif LOAD_CLEAN_DATA:
  print("Loading preprocessed data!")
  fpaths = list(CLEANED_DATA_DIR.glob("*.csv"))
  for p in tqdm(fpaths):
    fold_str, split_set, strategy = (p.name.split(".")[0]).split("_")
    if strategy in STRATEGIES_TO_LOAD:
      curr_df = pd.read_csv(p)
      fold = int(fold_str)
      if fold in data_dict.keys():
        if split_set in data_dict[fold].keys():
          data_dict[fold][split_set].update({strategy : curr_df})
        else:
          data_dict[fold][split_set] = {}
          data_dict[fold][split_set][strategy] = curr_df
      else:
        data_dict[fold] = {}
        data_dict[fold][split_set] = {}
        data_dict[fold][split_set][strategy] = curr_df
else:
  print("Skipped clean data loading!")


In [ ]:
%pwd

%ls -l data/cleaned/

In [ ]:
'Skipped clean data loading!' if not LOAD_CLEAN_DATA else data_dict[0][TRAIN][MISSING_VAL_STRATEGY].head()

In [ ]:
'Skipped clean data loading!' if not LOAD_CLEAN_DATA else data_dict[0][TEST][MISSING_VAL_STRATEGY].head()

Create RNN sequences

In [ ]:
def generate_rnn_sequences(df, feature_cols, label_col='SepsisLabel', group_col='patient_id', time_col='ICULOS', n=3):
    """
    For each patient, creates overlapping sequences of length `n` to predict the next label.

    Returns:
        sequences: list of (X_seq, y_next) pairs
    """
    sequences = []

    for _, group in df.groupby(group_col):
        #group = group.sort_values(by='index' if 'index' in group.columns else group.index)
        group = group.sort_values(by='index' if 'index' in group.columns else time_col)
        X = group[feature_cols].values
        y = group[label_col].values

        if len(X) <= n:
            continue  # skip short sequences

        for i in range(len(X) - n):
            X_seq = X[i:i+n]        # shape: (n, D)
            y_next = y[i+n]         # scalar: label at t+n
            sequences.append((X_seq, y_next))

    return sequences


## Feature Normalization and Addressing Class Imbalance

In [ ]:
REGENERATE_FOLDS = True

In [ ]:
# (Aparna) Run Feature Normalization and Addressing Class Imbalance codes here
import pickle
from pathlib import Path
import torch
from torch.utils.data import TensorDataset
from Data_Preparation import(
    train_validate_split,
    center,
    smote_oversample_to_tensor
)
import numpy as np
import joblib

#SEQ_LEN = 24
ID_COL = 'patient_id'
TIME_COL = 'ICULOS'
LABEL_COL = 'SepsisLabel'

PREPROCESSED_DATA_DIR = Path('data/preprocessed')
if not PREPROCESSED_DATA_DIR.is_dir():
    PREPROCESSED_DATA_DIR.mkdir()

if REGENERATE_FOLDS:

    for fold in range(5):
        FOLD_DIR = PREPROCESSED_DATA_DIR.joinpath('fold_' + str(fold))
        if not FOLD_DIR.is_dir():
            FOLD_DIR.mkdir()
        
        TRAIN_DIR = FOLD_DIR.joinpath('train')
        if not TRAIN_DIR.is_dir():
            TRAIN_DIR.mkdir()
            
        TEST_DIR = FOLD_DIR.joinpath('test')
        if not TEST_DIR.is_dir():
            TEST_DIR.mkdir()
        
        VAL_DIR = FOLD_DIR.joinpath('validate')
        if not VAL_DIR.is_dir():
            VAL_DIR.mkdir()
        
        print(f"\n=== Fold {fold} ===")
        train_df = data_dict[fold]['train']['impute'].copy()
        test_df = data_dict[fold]['test']['impute'].copy()
        print(f"Input: \nTrain shape: {train_df.shape}, Test shape: {test_df.shape}")

        #feature_cols = train_df.drop(columns=[ID_COL, TIME_COL, LABEL_COL]).columns
        col_mask = [ID_COL, TIME_COL, LABEL_COL]
        feature_cols = [x for x in train_df.columns.to_list() if not x in col_mask]
        
        # Apply standard scaler to center the data
        train_df[feature_cols] = center(train_df, feature_cols)
        test_df[feature_cols] = center(test_df, feature_cols)
        
        # k fold split for train/validate splitting within train set
        inner_splits = train_validate_split(train_df)
        for i_fold, (train_idx, val_idx) in enumerate(inner_splits):
            train_seqs = generate_rnn_sequences(train_df.iloc[train_idx], feature_cols)
            val_seqs = generate_rnn_sequences(train_df.iloc[val_idx], feature_cols)
            X_train_tensor, y_train_tensor = smote_oversample_to_tensor(
                np.array([x for x, y in train_seqs]), 
                np.array([y for x, y in train_seqs])
            )
            X_val_tensor = torch.tensor(np.array([x for x, y in val_seqs]), dtype=torch.float32)
            y_val_tensor = torch.tensor(np.array([y for x, y in val_seqs]), dtype=torch.float32)
            
            del train_seqs, val_seqs
            
            train_path = TRAIN_DIR.joinpath(f'compressed_train_dataset_{i_fold}.pkl.z')
            val_path = VAL_DIR.joinpath(f'compressed_val_dataset_{i_fold}.pkl.z')
            
            print( f"Writing data to {train_path}: ")
            joblib.dump(TensorDataset(X_train_tensor, y_train_tensor), train_path, compress=3)
            
            print( f"Writing data to {val_path}: ")
            joblib.dump(TensorDataset(X_val_tensor, y_val_tensor), val_path, compress=3)
            
            del X_train_tensor, y_train_tensor, X_val_tensor, y_val_tensor

        
        # RNN Sequences for test set
        test_sequences = generate_rnn_sequences(test_df, feature_cols)
        X_test_tensor = torch.tensor(np.array([x for x, y in test_sequences]), dtype=torch.float32)
        y_test_tensor = torch.tensor(np.array([y for x, y in test_sequences]), dtype=torch.float32)

        fname = 'compressed_test_dataset.pkl.z'
        path = TEST_DIR.joinpath(fname)
        print( f"Writing data to {path}: ")
        joblib.dump(TensorDataset(X_test_tensor, y_test_tensor), path, compress=3)
        print( f"Completed!! ")

        del test_sequences, X_test_tensor, y_test_tensor

## Exploratory data analysis (EDA)

In [ ]:
# Ehsan

!ls -l
############################################
###  Test Code Cell Please Don't Change  ###
############################################
# (Ehsan) Run Exploratory data analysis (EDA) codes here
# Lactate is the most relevant criteria then the rest of the plotted variables are most relevant
# 1. Serum Lactate
# 2. White Blood Cell Count (WBC)
# 3. Blood Urea Nitrogen (BUN) / Creatinine
# 4. Mean Arterial Pressure (MAP) / Systolic BP (SBP)
# 5. Heart Rate (HR) & Respiratory Rate (Resp)

from Data_Preparation import run_eda, run_comprehensive_eda
# Example:
#run_eda(data_A, ['HCO3','Lactate','WBC','BUN','MAP','HR','Resp'])
#run_eda(data_B, ['HCO3','Lactate','WBC','BUN','MAP','HR','Resp'])
#run_eda(data_A, ['Lactate','WBC'])
#run_eda(data_B, ['Lactate','WBC'])
############################################
######### A more comprehensive EDA #########
############################################
#run_comprehensive_eda
# 1) Missingness
# 2) Correlation heatmap (drop rows with any missing in features)
# 3) Boxplots for each feature by label
# 4) KDE overlays (all features in one grid)
# 5) PCA scatter
# Example:
if RUN_EDA_A:
    # 1) Automatically select all feature columns except the ones to drop:
    to_drop = ['SepsisLabel', 'patient_id', 'Unit1', 'Unit2', 'HospAdmTime']
    all_features = [col for col in data_A.columns if col not in to_drop]

    # 2) Quick sanity-check
    print("Running EDA on:", all_features)

    # 3) Call your comprehensive EDA (here we run all steps 1–5):
    from Data_Preparation.eda import run_comprehensive_eda
    run_comprehensive_eda(data_A, all_features, steps=[1,2])


##### Other examples
#run_comprehensive_eda(data_AB, ['HCO3','Lactate','WBC','BUN','MAP','HR','Resp','O2Sat','Temp','pH','PTT','Glucose','Chloride','Bilirubin_direct'], steps = [1,2])
#run_comprehensive_eda(data_A, ['HCO3','Lactate','WBC','BUN','MAP','HR','Resp'])
#run_comprehensive_eda(data_B, ['HCO3','Lactate','WBC','BUN','MAP','HR','Resp'])


from Data_Preparation.eda import corr_difference_analysis
if RUN_EDA_A:
    features = [c for c in data_A.columns
                if c not in ('SepsisLabel','patient_id','Unit1','Unit2','HospAdmTime')]

    diff_matrix, top_changes = corr_difference_analysis(
        data_A,
        features,
        min_count=50,   # only include features with ≥50 non‐null in each label
        top_k=15,
        figsize=(8,6)
    )


In [ ]:
# Ehsan
############################################
###      Ready to run Dataset_A EDA      ###
############################################
###  Please Don't Change  ###
from Data_Preparation import run_eda, run_comprehensive_eda, corr_difference_analysis
if RUN_EDA_A:
    features = [c for c in data_A.columns
                if c not in ('SepsisLabel','patient_id','Unit1','Unit2','HospAdmTime')]
    #run_comprehensive_eda(data_A)
    run_comprehensive_eda(data_A, all_features, steps=[1,2])
    #corr_difference_analysis(data_A)
    diff_matrix, top_changes = corr_difference_analysis(
        data_A,
        features,
        min_count=50,   # only include features with ≥50 non‐null in each label
        top_k=15,
        figsize=(8,6)
    )

In [ ]:
# Ehsan
############################################
###      Ready to run Dataset_B EDA      ###
############################################
###  Please Don't Change  ###
from Data_Preparation import run_eda, run_comprehensive_eda, corr_difference_analysis
if RUN_EDA_B:
    features = [c for c in data_B.columns
                if c not in ('SepsisLabel','patient_id','Unit1','Unit2','HospAdmTime')]
    #run_comprehensive_eda(data_B)
    run_comprehensive_eda(data_B, all_features, steps=[1,2])
    #corr_difference_analysis(data_B)
    diff_matrix, top_changes = corr_difference_analysis(
        data_B,
        features,
        min_count=50,   # only include features with ≥50 non‐null in each label
        top_k=15,
        figsize=(8,6)
    )

In [ ]:
# Ehsan
############################################
###      Ready to run Dataset_AB EDA     ###
############################################
###  Please Don't Change  ###
from Data_Preparation import run_eda, run_comprehensive_eda, corr_difference_analysis
if RUN_EDA_AB:
    features = [c for c in data_AB.columns
                if c not in ('SepsisLabel','patient_id','Unit1','Unit2','HospAdmTime')]
    #run_comprehensive_eda(data_AB)
    run_comprehensive_eda(data_AB, all_features, steps=[1,2])
    #corr_difference_analysis(data_AB)
    diff_matrix, top_changes = corr_difference_analysis(
        data_AB,
        features,
        min_count=50,   # only include features with ≥50 non‐null in each label
        top_k=15,
        figsize=(8,6)
    )

# Baseline Models

## Training - GRU

In [ ]:
import torch
import pickle
from pathlib import Path
from Training_Pipeline import train
from Model_Definitions import Baseline_GRU

LEARNING_RATE = 1e-4
INPUT_DIM = 39
HIDDEN_DIM = 64
LAYERS = 2
OUT_DIM = 2
EPOCHS = 10
SAVE = True
BASELINE_GRU_MODEL_DIR = Path('models/gru_baseline')

if not BASELINE_GRU_MODEL_DIR.is_dir():
    BASELINE_GRU_MODEL_DIR.mkdir(parents=True)

PREPROCESSED_DATA_DIR = Path('data/preprocessed')

baseline_gru_history_dict = {}

for fold in range(5):
    print( f"Reading alreay processed data for fold {fold}: ")
    fname = f'time_series_data_compressed_{fold}.pkl.z'
    preprocessed_fold = joblib.load(PREPROCESSED_DATA_DIR.joinpath(fname))
    model = Baseline_GRU(
                input_size=INPUT_DIM,
                hidden_size=HIDDEN_DIM,
                num_layers=LAYERS,
                output_size=OUT_DIM
            )
    optimizer = torch.optim.Adam(model.parameters(), LEARNING_RATE)
    criterion = torch.nn.CrossEntropyLoss()
    model, history = train(
        dataset_fold=preprocessed_fold,
        model=model,
        optimizer=optimizer,
        criterion=criterion,
        num_epochs=EPOCHS,
    )
    if SAVE:
        fname = 'baseline_gru_' + str(fold) + '.pt'
        torch.save(model.state_dict(), BASELINE_GRU_MODEL_DIR.joinpath(fname))
    baseline_gru_history_dict[fold] = history
    del preprocessed_fold
print('Finished!')

## Training - LSTM

In [ ]:
import torch
import pickle
from pathlib import Path
from Training_Pipeline import train
from Model_Definitions import Baseline_LSTM

LEARNING_RATE = 1e-4
INPUT_DIM = 39
HIDDEN_DIM = 64
LAYERS = 2
OUT_DIM = 2
EPOCHS = 10
SAVE = True
BASELINE_LSTM_MODEL_DIR = Path('models/lstm_baseline')

if not BASELINE_LSTM_MODEL_DIR.is_dir():
    BASELINE_LSTM_MODEL_DIR.mkdir(parents=True)

PREPROCESSED_DATA_DIR = Path('data/preprocessed')

baseline_lstm_history_dict = {}


for fold in range(5):
    print( f"Reading alreay processed data for fold {fold}: ")
    fname = f'time_series_data_compressed_{fold}.pkl.z'
    preprocessed_fold = joblib.load(PREPROCESSED_DATA_DIR.joinpath(fname))
    model = Baseline_LSTM(
                input_size=INPUT_DIM,
                hidden_size=HIDDEN_DIM,
                num_layers=LAYERS,
                output_size=OUT_DIM
            )
    optimizer = torch.optim.Adam(model.parameters(), LEARNING_RATE)
    criterion = torch.nn.CrossEntropyLoss()
    model, history = train(
                        dataset_fold=preprocessed_fold,
                        model=model,
                        optimizer=optimizer,
                        criterion=criterion,
                        num_epochs=EPOCHS,
                    )
    if SAVE:
        fname = 'baseline_lstm_' + str(fold) + '.pt'
        torch.save(model.state_dict(), BASELINE_LSTM_MODEL_DIR.joinpath(fname))
    baseline_lstm_history_dict[fold] = history
    del preprocessed_fold
print('Finished!')